# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117226e+02     1.647761e+00
 * time: 0.14574480056762695
     1     1.047858e+01     9.330579e-01
 * time: 1.3596558570861816
     2    -1.192519e+01     1.068854e+00
 * time: 1.4939439296722412
     3    -3.408713e+01     7.551034e-01
 * time: 1.657696008682251
     4    -4.756239e+01     5.737868e-01
 * time: 1.8132150173187256
     5    -5.706476e+01     2.041715e-01
 * time: 1.9594519138336182
     6    -5.998152e+01     1.429440e-01
 * time: 2.0698180198669434
     7    -6.105287e+01     4.986470e-02
 * time: 2.1849420070648193
     8    -6.144946e+01     7.090447e-02
 * time: 2.2938008308410645
     9    -6.170875e+01     3.599303e-02
 * time: 2.401798963546753
    10    -6.190076e+01     2.838875e-02
 * time: 2.516918897628784
    11    -6.204229e+01     1.887185e-02
 * time: 2.624202013015747
    12    -6.209666e+01     1.545298e-02
 * time: 2.7393457889556885
    13    -6.214787e+01     1.679045e-02
 * time: 2.8477859497070

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557687
    AtomicNonlocal      14.8522660
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666462290
